# Explore harvested text files

In [122]:
import os
import pandas as pd
import fileinput
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
from operator import itemgetter
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /Users/tim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def get_latest_harvest():
    '''
    Get the timestamp of the most recent harvest.
    '''
    harvests = sorted([d for d in os.listdir('data') if os.path.isdir(os.path.join('data', d))])
    return harvests[-1]

def open_harvest_data(timestamp=None):
    '''
    Open the results of the specified harvest (most recent by default).
    
    Returns a DataFrame.
    '''
    if not timestamp:
        timestamp = get_latest_harvest()
    print(timestamp)
    df = pd.read_csv(os.path.join('data', timestamp, 'results.csv'), parse_dates=['date'])
    return df 

## Aggregate the text files

In [120]:
def aggregate_texts(timestamp=None):
    '''
    Aggregate all individual article texts creating one big file.
    '''
    if not timestamp:
        timestamp = get_latest_harvest()
    output_file = os.path.join('data', timestamp, 'all-texts.txt')
    data_dir = os.path.join('data', timestamp, 'text')
    files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file[-4:] == '.txt']                                                                                   
    with open(output_file, 'w') as fout, fileinput.input(files) as fin:
        for line in fin:
            fout.write(line)
    
def aggregate_years(timestamp=None):
    '''
    Aggregate individual article text by year, creating one file per year.
    '''
    if not timestamp:
        timestamp = get_latest_harvest()
    output_dir = os.path.join('data', timestamp, 'years')
    os.makedirs(output_dir, exist_ok=True)
    data_dir = os.path.join('data', timestamp, 'text')
    df = open_harvest_data(timestamp=timestamp)
    df['year'] = df['date'].dt.year
    years = list(df['year'].unique())
    for year in years:
        output_file = os.path.join(output_dir, '{}.txt'.format(year))
        files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file[-4:] == '.txt' and file[:4] == str(year)]                                                                                   
        with open(output_file, 'w') as fout, fileinput.input(files) as fin:
            for line in fin:
                fout.write(line)
                
                
def aggregate_newspapers(timestamp=None):
    '''
    Aggregate individual article text by newspaper, creating one file per newspaper.
    '''
    if not timestamp:
        timestamp = get_latest_harvest()
    output_dir = os.path.join('data', timestamp, 'newspapers')
    os.makedirs(output_dir, exist_ok=True)
    data_dir = os.path.join('data', timestamp, 'text')
    df = open_harvest_data(timestamp=timestamp)
    newspapers = list(df['newspaper_id'].unique())
    for newspaper in newspapers:
        output_file = os.path.join(output_dir, '{}.txt'.format(newspaper))
        files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file[-4:] == '.txt' and '-{}-'.format(newspaper) in file]                                                                                   
        with open(output_file, 'w') as fout, fileinput.input(files) as fin:
            for line in fin:
                fout.write(line)

## Display word frequencies

In [126]:
def show_word_frequencies(text_file):
    with open(text_file, 'r') as text:
        blob = TextBlob(text.read())
    word_counts = [[word, count] for word, count in blob.lower().word_counts.items() if word not in stopwords]
    word_counts = sorted(word_counts, key=itemgetter(1), reverse=True)[:25]
    return pd.DataFrame(word_counts).style.format({1: '{:,}'}).bar(subset=[1], color='#d65f5f').set_properties(subset=[1], **{'width': '300px'})

def word_frequency_all(timestamp=None):
    if not timestamp:
        timestamp = get_latest_harvest()
    data_dir = os.path.join('data', timestamp)
    return show_word_frequencies(os.path.join(data_dir, 'all-texts.txt'))

def word_frequency_by_year(year, timestamp=None):
    if not timestamp:
        timestamp = get_latest_harvest()
    data_dir = os.path.join('data', timestamp, 'years')
    return show_word_frequencies(os.path.join(data_dir, '{}.txt'.format(year)))

def word_frequency_by_newspaper(newspaper_id, timestamp=None):
    if not timestamp:
        timestamp = get_latest_harvest()
    data_dir = os.path.join('data', timestamp, 'newspapers')
    return show_word_frequencies(os.path.join(data_dir, '{}.txt'.format(year)))   

In [121]:
aggregate_texts()

In [127]:
word_frequency_all()

,0,1
0,mr,"17,442"
1,tho,"15,989"
2,south,"7,590"
3,one,"7,536"
4,would,"7,109"
5,weather,"6,983"
6,j,"6,684"
7,new,"6,451"
8,«,"5,795"
9,—,"5,269"


In [73]:
def calculate_tfidf(names, files, ngram_size=1):
    # Chomp chomp -- getting trigrams
    tf = TfidfVectorizer(input='filename', analyzer='word', ngram_range=(ngram_size, ngram_size), min_df=0, smooth_idf=False, sublinear_tf=True)
    tfidf_matrix = tf.fit_transform(files)
    # These are the actual phrases
    feature_names = tf.get_feature_names()
    # These are the scores
    texts = tfidf_matrix.todense()
    for index, row in enumerate(texts):
        name = names[index]
        print('\n\n{}\n'.format(name.upper()))
        text = row.tolist()[0]
        # If the score is not 0 save it with an index (which will let us get the feature_name)
        scores = [pair for pair in zip(range(0, len(text)), text) if pair[1] > 0]
        sorted_scores = sorted(scores, key=lambda t: t[1] * -1)
        # Print the top 20 results for each file
        for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_scores][:20]:
            print('{0: <40} {1}'.format(phrase, score))

def calculate_tfidf_by_year(timestamp=None, ngram_size=1):
    if not timestamp:
        timestamp = get_latest_harvest()
    data_dir = os.path.join('data', timestamp, 'years')
    if not os.path.exists(data_dir):
        aggregate_years(timestamp)
    # Get a list of the file names in the directory.
    names = [file[:-4] for file in os.listdir(data_dir) if file[-4:] == '.txt']
    # Get a list of files to feed to scikit-learn.
    files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file[-4:] == '.txt']
    calculate_tfidf(names, files, ngram_size)
    
    
def calculate_tfidf_by_newspaper(timestamp=None, ngram_size=1):
    if not timestamp:
        timestamp = get_latest_harvest()
    data_dir = os.path.join('data', timestamp, 'newspapers')
    if not os.path.exists(data_dir):
        aggregate_newspapers(timestamp)
    df = open_harvest_data(timestamp=timestamp)
    newspapers = df[['newspaper_id', 'newspaper_title']].drop_duplicates().set_index('newspaper_id')
    # Get a list of the file names in the directory.
    names = [newspapers.loc[int(file[:-4])]['newspaper_title'] for file in os.listdir(data_dir) if file[-4:] == '.txt']
    # Get a list of files to feed to scikit-learn.
    files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file[-4:] == '.txt']
    calculate_tfidf(names, files, ngram_size)

In [106]:
calculate_tfidf_by_newspaper(ngram_size=1)

1536200911


THE MERCURY (HOBART, TAS. : 1860 - 1954)

rometch                                  0.0258397352435745
strahan                                  0.025216227810368565
brownell                                 0.02342578177640727
mongolia                                 0.02342578177640727
pedder                                   0.02342578177640727
launceston                               0.022807321553748865
kingsmill                                0.022763180065568576
giblin                                   0.022245718285806303
agst                                     0.022166077209062728
bellerive                                0.021895913229468662
fingal                                   0.021895913229468662
mathinna                                 0.02174154644119813
bennison                                 0.021081666287104856
wherrett                                 0.020798784925900708
oonah                                    0.020668396741229236
corinna              

ibsen                                    0.16188636859243138
outis                                    0.12731560062574437
gong                                     0.09021494307751633
gissing                                  0.08835636408992066
persevering                              0.07536903817968268
clocks                                   0.07278484803546735
socialistic                              0.0682063210311603
ashton                                   0.06703786476798426
classical                                0.06554450539271246
tho                                      0.06422839406726136
fisheries                                0.06182580553980686
acclimatised                             0.057987219306251454
ainuso                                   0.057987219306251454
antago                                   0.057987219306251454
bescecliod                               0.057987219306251454
btrngglers                               0.057987219306251454
caricaturist        

cools                                    0.12324843963459717
aainrt                                   0.11351331819157744
atuiwphere                               0.11351331819157744
eanpci                                   0.11351331819157744
enargy                                   0.11351331819157744
ensrgy                                   0.11351331819157744
ffiond                                   0.11351331819157744
grsat                                    0.11351331819157744
jify                                     0.11351331819157744
lozy                                     0.11351331819157744
prohably                                 0.11351331819157744
raiulfthpf                               0.11351331819157744
remember1                                0.11351331819157744
simnly                                   0.11351331819157744
sotur                                    0.11351331819157744
suspsnse                                 0.11351331819157744
tasuiuuia               

spluttered                               0.0899607828502847
cooled                                   0.0853612253183565
condenser                                0.08492100387451332
changing                                 0.08485137854972304
moods                                    0.08330256490799366
energies                                 0.08051592687679222
milky                                    0.07511733464163328
suns                                     0.07448748190159223
cracked                                  0.07390867808464079
earth                                    0.07284907680806862
sun                                      0.07202042540353049
860000                                   0.07167117586835571
betweeri                                 0.07167117586835571
coinuni                                  0.07167117586835571
cotiipared                               0.07167117586835571
fiftth                                   0.07167117586835571
greatey                   



VICTORIAN EXPRESS (GERALDTON, WA : 1878 - 1894)

balmaceda                                0.14663624975044467
insurgents                               0.12075743124496181
loyalists                                0.10870645404634782
santiago                                 0.09291831949256939
chilian                                  0.09274658566658935
placilla                                 0.08893199989807862
paraiso                                  0.08003296376577419
surgents                                 0.08003296376577419
valparaiso                               0.07926344334867201
8st                                      0.07113392763346979
lltli                                    0.07113392763346979
subsiding                                0.07068633384798571
kintore                                  0.06394924701048535
fled                                     0.061040829373574565
rifles                                   0.05965620793868388
bedford                          

...

likened                                  0.04904722459675186
installation                             0.04839026646641298
messina                                  0.04752067156870505
planets                                  0.04661501451010947
outbursts                                0.04533315937572297
wrugge                                   0.04533315937572297
solar                                    0.0451175226729276


TASMANIAN NEWS (HOBART, TAS. : 1883 - 1911)

pills                                    0.05259199523924706
obart                                    0.0502980139648862
rth                                      0.04745127982358123
ellington                                0.04509231126400948
kingsmiil                                0.04509231126400948
onowai                                   0.04509231126400948
rifted                                   0.04509231126400948
ith                                      0.04442976010516395
pink                                    

cnristian                                0.1431319215205729
cojirse                                  0.1431319215205729
dardanelia                               0.1431319215205729
ftreat                                   0.1431319215205729
hifetorv                                 0.1431319215205729
jafter                                   0.1431319215205729
jiassen                                  0.1431319215205729
jjiat                                    0.1431319215205729
kunberley                                0.1431319215205729
leadens                                  0.1431319215205729
mafcking                                 0.1431319215205729
pfenoa                                   0.1431319215205729
recorfla                                 0.1431319215205729
rrfjftntfid                              0.1431319215205729
tearie                                   0.1431319215205729
ueii                                     0.1431319215205729
ueonta                                  

1914                                     0.09830407222272677
aailed                                   0.09541764692288762
atarvatioo                               0.09541764692288762
basod                                    0.09541764692288762
bbokiuhd                                 0.09541764692288762
bcllevu                                  0.09541764692288762
bnrwood                                  0.09541764692288762
demolishrd                               0.09541764692288762
drewned                                  0.09541764692288762
earlli                                   0.09541764692288762
ebu                                      0.09541764692288762
fallb                                    0.09541764692288762
fuuetionary                              0.09541764692288762
goast                                    0.09541764692288762
iniuiuium                                0.09541764692288762
inufct                                   0.09541764692288762
neariug                 

leoturer                                 0.07751157615785978
wbioh                                    0.05960955440901772
wae                                      0.05688002049428971
aod                                      0.05352083713102781
glblin                                   0.052946762686165626
inhabitated                              0.052946762686165626
thamae                                   0.052946762686165626
zino                                     0.052946762686165626
iters                                    0.052492304695795314
oot                                      0.04879029392474487
davonport                                0.04764861179815309
obarge                                   0.04764861179815309
scientifio                               0.04764861179815309
ulver                                    0.04764861179815309
comet                                    0.04623913229078214
beiog                                    0.04454939220550328
formby             

accomodation                             0.13222959004453994
tiiany                                   0.13222959004453994
fdr                                      0.09802630576141422
drama                                    0.0955072624968795
colombo                                  0.07814576735194953
6olumns                                  0.0780969259865583
7l0                                      0.0780969259865583
anetan                                   0.0780969259865583
asocial                                  0.0780969259865583
attaioing                                0.0780969259865583
aused                                    0.0780969259865583
baxaar                                   0.0780969259865583
bitnddfy                                 0.0780969259865583
botelkeepert                             0.0780969259865583
brfr                                     0.0780969259865583
bubgay                                   0.0780969259865583
cdntafos                            

In [74]:
aggregate_newspapers()

1536200911


In [75]:
df = open_harvest_data()

1536200911


In [99]:
newspapers = df[['newspaper_id', 'newspaper_title']].drop_duplicates().set_index('newspaper_id')
newspapers.loc['10']

KeyError: 'the label [10] is not in the [index]'

In [85]:
n

,,article_id,words,corrections
newspaper_id,newspaper_title,,,
7,"The South Australian Advertiser (Adelaide, SA : 1858 - 1889)",686068440,33488,16
8,The Maitland Mercury and Hunter River General Advertiser (NSW : 1843 - 1893),75959376,5528,0
9,"Northern Territory Times and Gazette (Darwin, NT : 1873 - 1927)",31281505,19437,13
10,"The Mercury (Hobart, Tas. : 1860 - 1954)",322427246,88715,31
11,The Canberra Times (ACT : 1926 - 1995),713157939,5589,4
12,"The Courier-Mail (Brisbane, Qld. : 1933 - 1954)",76516085,1271,1
13,"The Argus (Melbourne, Vic. : 1848 - 1957)",196460675,122907,80
16,The Brisbane Courier (Qld. : 1864 - 1933),2020246731,401596,397
30,"The West Australian (Perth, WA : 1879 - 1954)",420072286,23005,9
